In [ ]:
# Cell 1: توابع پیش‌پردازش (می‌توان از src/preprocessing.py import کرد)
def handle_missing_values(df, strategy='mean'):
    """مدیریت مقادیر گمشده"""
    from sklearn.impute import SimpleImputer
    
    # تشخیص ستون‌های عددی و غیرعددی
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns
    
    if strategy == 'mean':
        imputer = SimpleImputer(strategy='mean')
    elif strategy == 'median':
        imputer = SimpleImputer(strategy='median')
    elif strategy == 'mode':
        imputer = SimpleImputer(strategy='most_frequent')
    
    df_numeric = pd.DataFrame(
        imputer.fit_transform(df[numeric_cols]),
        columns=numeric_cols
    )
    
    # برای ستون‌های غیرعددی
    df_categorical = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])
    
    return pd.concat([df_numeric, df_categorical], axis=1)


In [ ]:
# Cell 2: اجرای مدیریت مقادیر گمشده
df_clean = handle_missing_values(df, strategy='mean')
print("Missing values after cleaning:", df_clean.isnull().sum().sum())


In [ ]:
# Cell 3: حذف ویژگی‌های با واریانس کم
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.01)  # حذف ویژگی‌های با واریانس کمتر از 0.01
X = df_clean.drop('label', axis=1)
y = df_clean['label']

X_selected = selector.fit_transform(X)
print(f"Original features: {X.shape[1]}")
print(f"Selected features: {X_selected.shape[1]}")


In [ ]:
# Cell 4: ذخیره شاخص ویژگی‌های انتخاب شده
selected_indices = selector.get_support(indices=True)
np.save('../data/selected_feature_indices.npy', selected_indices)


In [ ]:
# Cell 5: مقیاس‌بندی
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# نمایش توزیع قبل و بعد از مقیاس‌بندی
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(X_selected[:, 0], bins=50, alpha=0.7)
axes[0].set_title('Before Scaling')
axes[1].hist(X_scaled[:, 0], bins=50, alpha=0.7, color='orange')
axes[1].set_title('After Scaling')
plt.show()


In [ ]:
# Cell 6: کدگذاری برچسب‌ها
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Label mapping:")
for i, label in enumerate(le.classes_):
    print(f"{label}: {i}")


In [ ]:
# Cell 7: ذخیره encoder
import joblib
joblib.dump(le, '../models/label_encoder.pkl')
joblib.dump(scaler, '../models/scaler.pkl')
